In [1]:
import numpy as np
import torch
import random

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [2]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer

splits = load_dataset("imdb", split=["train", "test"])
dataset = DatasetDict(train=splits[0], test=splits[1])
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Using the latest cached version of the dataset since imdb couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at C:\Users\sazgar\.cache\huggingface\datasets\imdb\plain_text\0.0.0\e6281661ce1c48d982bc483cf8a173c1bbeb5d31 (last modified on Sun Dec  8 19:09:31 2024).
c:\Users\sazgar\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x["text"], return_tensors="pt", padding=True, max_length=512, truncation=True
    ),
    batched=True
)
tokenized_dataset.set_format(type="torch")

In [4]:
target_subset = tokenized_dataset["train"].train_test_split(
    test_size=0.5,
    stratify_by_column="label"
)

In [5]:
N_SHADOWS = 10

shadow_subsets = []
for i in range(N_SHADOWS):
    shadow_subsets.append(
        tokenized_dataset["test"].train_test_split(test_size=0.5, stratify_by_column="label")
    )

# Train Target

In [9]:
from transformers import BertConfig, BertForSequenceClassification
from model.bert_configs import target, shadow
target_bert_config = BertConfig(**shadow)
target_classifier = BertForSequenceClassification(config=target_bert_config)

In [ ]:
import evaluate
def compute_metrics(eval_preds):
    accuracy = evaluate.load("accuracy")
    logits, labels= eval_preds
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [10]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="saved_models/victim_small",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

trainer = Trainer(
    model=target_classifier,
    args=training_args,
    train_dataset=target_subset["train"],
    eval_dataset=target_subset["test"]
)

In [11]:
trainer.train()

  0%|          | 0/1960 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Train Shadows

# Train Attacker